In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential 
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from skimage.transform import resize
import h5py
import os


In [2]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Resize images to 42x42
x_train_resized = np.zeros((x_train.shape[0], 42, 42))
for i in range(x_train.shape[0]):
    x_train_resized[i] = resize(x_train[i], (42, 42))

x_test_resized = np.zeros((x_test.shape[0], 42, 42))
for i in range(x_test.shape[0]):
    x_test_resized[i] = resize(x_test[i], (42, 42))

# Reshape to add channel dimension (required for Keras)
x_train_resized = x_train_resized.reshape(-1, 42, 42, 1)
x_test_resized = x_test_resized.reshape(-1, 42, 42, 1)

# Normalize pixel values to range [0, 1]
x_train_resized = x_train_resized.astype('float32') / 255.0
x_test_resized = x_test_resized.astype('float32') / 255.0

# One-hot encode labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [4]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(4, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model (Replace X_train and y_train with your training data)
model.fit(x_train_resized, y_train, batch_size=50, epochs=15, validation_data=(x_test_resized, y_test))

# Save the model's weights and biases
model.save('model_weights.h5')


Epoch 1/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4164 - loss: 1.7995 - val_accuracy: 0.8563 - val_loss: 0.5345
Epoch 2/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7707 - loss: 0.7177 - val_accuracy: 0.8705 - val_loss: 0.4622
Epoch 3/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7886 - loss: 0.6700 - val_accuracy: 0.8808 - val_loss: 0.4328
Epoch 4/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7966 - loss: 0.6558 - val_accuracy: 0.8811 - val_loss: 0.4280
Epoch 5/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7968 - loss: 0.6482 - val_accuracy: 0.8823 - val_loss: 0.4246
Epoch 6/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8040 - loss: 0.6307 - val_accuracy: 0.8860 - val_loss: 0.4117
Epoch 7/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8057 - loss: 0.6265 - val_accuracy: 0.8837 - val_loss: 0.4215
Epoch 8/15
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8111 - loss: 0.6119 - 

In [5]:
from keras.models import load_model

# Load the model from the .h5 file
model = load_model('model_weights.h5')

# Get the weights for each layer
weights = model.get_weights()

# Save the weights to a text file
with open('model_weights.txt', 'w') as file:
    for layer_weights in weights:
        for weight in layer_weights.flatten():
            file.write(str(weight) + '\n')

print("Model weights saved to model_weights.txt")

Model weights saved to model_weights.txt


In [6]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 40, 40, 4)      │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 4)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 18, 18, 4)      │           148 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 9, 9, 4)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 324)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 324)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         3,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,440 (13.44 KB)

 Trainable params: 3,438 (13.43 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [9]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.callbacks import ModelCheckpoint

# model = Sequential()
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(10, activation='softmax'))

# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model = load_model('model_weights.h5')

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test_resized, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step - accuracy: 0.9181 - loss: 0.2984
Test Loss: 0.2572062909603119
Test Accuracy: 0.9316999912261963


In [17]:
from keras.preprocessing import image
import numpy as np

# Load and preprocess the single image
img_path = 'demo7.png'  
img = image.load_img(img_path, target_size=(42, 42), color_mode='grayscale')  
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize the pixel values

# Predict the output of the model for the single image
predictions = model.predict(img_array)

# Print the predicted class probabilities
print("Predicted class probabilities:", predictions)

# Get the predicted class label
predicted_class = np.argmax(predictions)
print("Predicted class label:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted class probabilities: [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
Predicted class label: 3
